### IMPORTANT:
Before the following code, navigate to the layout window in Cadence, and open the EMX gui.
Select the desired settings before continuing. Also, define a custom function in cadence for closing the
dialog box in extract s params:

procedure(CloseBox()
hiDBoxOK(EMX_dialog_box)
);


EMX:
Process: 9HP_2023_Process/422OLLD.proc

Black Box Cells: if you were to have transistors/other blocks around that you cant simulate

Accuracy options:
	- divides the design into finite elements
		1, 1, 0.5, *
		for these options


Ports:
	signals: LEFT RIGHT BOTTOM TOP LEFT_m RIGHT_m BOTTOM_m TOP_m

Start:
	30e9
Stop
	100e9
Step
	1e9

Advanced Options:
	- make Label depth = 0 
	- cadence pins checked



In [3]:
import time, os, shutil
from EMXSimulator3 import EMXSimulator as sim

# figure out which window number the emx window will pop up as in order to close after running a test
EMX_WINDOW_NUMBER=9439
# starts at 3 in new cadence instance

# Run 1000 emx simulations, deleting and reinitializing a random structure each time
trials = 10000
# for naming files, make sure to update to start at most recent
# i.e. if there are 10000 data points in the results already, start at 10000
start_num = 20000

# desired structure size (roughly the length of square side) in microns
STRUCT_SIZE = (200, 200)

# name of layout cell
LIBRARY_NAME = "Thesis"
CELL_NAME = "test2"

# EMX work folder path
EMX_WORK_PATH = "/home/sbfisher/Cadence/9HPsetup/EMX_work/"

# directory to store results
RESULTS_DIR = "/scratch/gpfs/sbfisher/Test_Results300/"

# static left, right, bottom, top pins
# STATIC_PINS = [(8,0),(8,17),(17,7),(0,7)]

# Initialize Simulator
Simulator = sim("server-2", struct_width=STRUCT_SIZE[0], struct_height=STRUCT_SIZE[1])
# create first random structure
Simulator.create_random_structure()
# Create pins
Simulator.create_pins()
# Simulator.create_pins(pin_locations=[(8,0),(8,17),(17,7),(0,7)])

for i in range(trials):
    # delete current sparam file if it exists
    sparam_file = EMX_WORK_PATH + "{}_{}.work/{}.s8p".format(LIBRARY_NAME, CELL_NAME, CELL_NAME)
    if os.path.exists(sparam_file):
        os.remove(sparam_file)
    
    print("Starting Simulation #{}. ".format(i))
    t = time.localtime()
    current_time = time.strftime("%H:%M:%S", t)
    print(current_time)
    
    # run emx, creates sparam file
    Simulator.run_emx()

    # wait for s_param_file to be 1139 lines (kinda hacky, but this ensures that the simulation has completed
    line_count = 0
    while line_count < 1139:
        time.sleep(1) # give emx time to create sparam file. Might be better way if this ends up failing
        try:
            s_param_file = open(sparam_file, "r")
            lines = s_param_file.readlines()
            line_count = len(lines)
            s_param_file.close()
        except FileNotFoundError:
            print("s param file not created yet. Failing gracefully")
        # else:
        #     print("other error when reading sparam file")

    
    # move completed sparam file to data folder
    shutil.move(sparam_file, RESULTS_DIR + "trial_{}_data.s8p".format(i+start_num));
     # Create a file with the form of the structure and store
    Simulator.create_struct_file(RESULTS_DIR + "trial_{}_struct.txt".format(i+start_num))

    # delete current random structre and create new one
    Simulator.delete_structure()
    Simulator.delete_pins()
    Simulator.create_random_structure()
    Simulator.create_pins()

    # Simulator.create_pins(pin_locations=[(8,0),(8,17),(17,7),(0,7)])

    # Close EMX sim window
    Simulator.close_window(EMX_WINDOW_NUMBER+i)
    
    time.sleep(0.5) # try to fix job currently running error
    # Figure out which window number you are currently on for this
    print("Completed Simulation #{}".format(i))
    t = time.localtime()
    current_time = time.strftime("%H:%M:%S", t)
    print(current_time)
    print()

Simulator.delete_structure()
Simulator.delete_pins()

Starting Simulation #0. 
15:34:00
Completed Simulation #0
15:34:10

Starting Simulation #1. 
15:34:10
Completed Simulation #1
15:34:19

Starting Simulation #2. 
15:34:19
Completed Simulation #2
15:34:27

Starting Simulation #3. 
15:34:27
Completed Simulation #3
15:34:34

Starting Simulation #4. 
15:34:34
Completed Simulation #4
15:34:44

Starting Simulation #5. 
15:34:44
Completed Simulation #5
15:34:52

Starting Simulation #6. 
15:34:52
Completed Simulation #6
15:35:00

Starting Simulation #7. 
15:35:00
Completed Simulation #7
15:35:09

Starting Simulation #8. 
15:35:09
Completed Simulation #8
15:35:17

Starting Simulation #9. 
15:35:17
Completed Simulation #9
15:35:25

Starting Simulation #10. 
15:35:25
Completed Simulation #10
15:35:36

Starting Simulation #11. 
15:35:36
Completed Simulation #11
15:35:43

Starting Simulation #12. 
15:35:43
Completed Simulation #12
15:35:52

Starting Simulation #13. 
15:35:52
Completed Simulation #13
15:36:01

Starting Simulation #14. 
15:36:01
Compl

RuntimeError: ("error" 0 t nil ("*Error* A job is currently running"))
